### Retrieve global coins blockchain data

In [271]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockchain data:')
# print(df)
global_df

Global blockchain data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,584.0000,3.125000,3.462442e-03,SHA-256,2.976828,147.945205,0.010820,1.600786,0.537749
BTC,587.0000,3.157856,1.000000e+00,SHA-256,835.340194,147.189097,3.157856,464.802009,0.556422
DOGE,63.0000,10000.000000,2.689000e-06,Scrypt,0.001986,1371.428571,0.026890,36.877714,18565.529796
KAS,0.9995,65.449525,9.923429e-07,kHeavyHash,1.441147,86443.221611,0.000065,5.614346,3.895747
LTC,160.0000,6.250000,1.259000e-03,Scrypt,0.002049,540.000000,0.007869,4.249125,2073.570476
RVN,60.0000,2500.000000,1.500000e-07,KawPow,0.000006,1440.000000,0.000375,0.540000,96897.212158


### Retrieve NiceHash marketplace algorithms data

In [272]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['paying_btc_eh_day_$'] = (nh_market_df.paying_btc_eh_day * btc_exchange_rate).map('${:,.2f}'.format)
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

Nicehash marketplace data:


,hashrate_eh_s,paying_btc_eh_day,paying_btc_eh_day_$,daily_consumption_btc
SCRYPT,1.321999e-04,16992.063921,"$1,635,972,975.06",2.246349
SHA256ASICBOOST,7.365915e+00,0.539923,"$51,983.07",3.977028
KAWPOW,7.446064e-07,99281.036820,"$9,558,644,195.62",0.073925
KHEAVYHASH,5.483484e-02,3.839223,"$369,635.22",0.210523


### Calculate Nicehash max daily and max hourly consumption rate

In [273]:
max_market_utilisation = 0.76

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_$'] = (nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate).map('${:,.2f}'.format)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_$
SCRYPT,1.004719e-04,1.707225,0.071134,"$6,848.72"
SHA256ASICBOOST,5.598095e+00,3.022541,0.125939,"$12,125.26"
KAWPOW,5.659009e-07,0.056183,0.002341,$225.39
KHEAVYHASH,4.167448e-02,0.159998,0.006667,$641.85


### Calculate Nicehash hashrate to global hashrate ratio
Cap Nicehash hashrate to 15% of global hashrate

In [274]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']
reachable_df['nh_to_global_ratio_capped'] = reachable_df['nh_hashrate_eh_s_capped'] / reachable_df['nethash_eh_s']  

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_hashrate_eh_s_capped,nh_to_global_ratio_capped
BCH,SHA-256,2.976828,5.598095e+00,1.880557,4.465242e-01,0.150000
BTC,SHA-256,835.340194,5.598095e+00,0.006702,5.598095e+00,0.006702
DOGE,Scrypt,0.001986,1.004719e-04,0.050581,1.004719e-04,0.050581
KAS,kHeavyHash,1.441147,4.167448e-02,0.028918,4.167448e-02,0.028918
LTC,Scrypt,0.002049,1.004719e-04,0.049030,1.004719e-04,0.049030
RVN,KawPow,0.000006,5.659009e-07,0.101545,5.659009e-07,0.101545


### Calculate burn rate needed for specific amount to be spent in the time period
```
B - burn rate
A - amount
P - price
D - duration

B = A / P * D
```

In [275]:
min_order_amount_btc = 0.0001 # BTC
order_duration_s = 3600  # 1 hour

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['amount_btc'] = min_order_amount_btc
speed_df['duration_s'] = order_duration_s
speed_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
speed_df['speed_eh_s'] = speed_df.amount_btc / (speed_df.paying_btc_eh_s * speed_df.duration_s)
speed_df

,amount_btc,duration_s,paying_btc_eh_s,speed_eh_s
SCRYPT,0.0001,3600,0.196667,1.412424e-07
SHA256ASICBOOST,0.0001,3600,0.000006,4.445077e-03
KAWPOW,0.0001,3600,1.149086,2.417380e-08
KHEAVYHASH,0.0001,3600,0.000044,6.251265e-04


### Calculate amount needed to buy speed for the time period
```
B - burn rate
A - amount
P - price
D - duration

A = B * P * D 
```

In [276]:
speed_eh_s = 1 # EH / s
order_duration_s = 3600  # 1 hour

amount_df = pd.DataFrame(index=nh_market_df.index)
amount_df['speed_eh_s'] = speed_eh_s
amount_df['duration_s'] = order_duration_s
amount_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
amount_df['amount_btc'] = amount_df.speed_eh_s * amount_df.paying_btc_eh_s * amount_df.duration_s
amount_df

,speed_eh_s,duration_s,paying_btc_eh_s,amount_btc
SCRYPT,1,3600,0.196667,708.002663
SHA256ASICBOOST,1,3600,0.000006,0.022497
KAWPOW,1,3600,1.149086,4136.709867
KHEAVYHASH,1,3600,0.000044,0.159968


### Calculate max RTP for the coin on the NiceHash market for 1H duration

In [309]:
import pandas as pd
import numpy as np

max_allowed_coin_market_hp_share = 0.15
max_nicehash_market_utilisation = 0.76
speed_eh_s = 1
order_duration_s = 3600
order_amount_btc = 1

rtp_df = pd.DataFrame()
# rtp_df.loc['BCH', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BCH', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['BTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['DOGE', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['DOGE', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['LTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['LTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['KAS', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KHEAVYHASH', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['KAS', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['RVN', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KAWPOW', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['RVN', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df['nh_to_global_ratio'] = rtp_df['nh_hashrate_eh_s'] / global_df['nethash_eh_s']
# rtp_df['max_global_btc_reward_h'] = rtp_df.nh_to_global_ratio * global_df['daily_emmisions_btc'] / 24
# rtp_df['max_global_btc_reward_h'] = global_df['daily_emmisions_btc'] / 24
rtp_df['global_price_btc_eh_s'] = global_df['index_price_btc_eh_day'] / 86400
rtp_df.loc['BCH', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['BTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['DOGE', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
rtp_df.loc['LTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
rtp_df.loc['KAS', 'nh_price_btc_eh_s'] = nh_market_df.loc['KHEAVYHASH', 'paying_btc_eh_day'] / 86400
rtp_df.loc['RVN', 'nh_price_btc_eh_s'] = nh_market_df.loc['KAWPOW', 'paying_btc_eh_day'] / 86400
rtp_df['global_1eh_1h_duration_btc'] = rtp_df.global_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['nh_1eh_1h_duration_btc'] = rtp_df.nh_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['apy_%'] =  (rtp_df.global_1eh_1h_duration_btc / rtp_df.nh_1eh_1h_duration_btc * 100).map('{:,.2f}'.format)

rtp_df


,global_price_btc_eh_s,nh_price_btc_eh_s,global_1eh_1h_duration_btc,nh_1eh_1h_duration_btc,apy_%
BCH,0.000006,0.000006,0.022406,0.022497,99.60
BTC,0.000006,0.000006,0.023184,0.022497,103.06
DOGE,0.214879,0.196667,773.563741,708.002663,109.26
KAS,0.000045,0.000044,0.162323,0.159968,101.47
LTC,0.024000,0.196667,86.398770,708.002663,12.20
RVN,1.121496,1.149086,4037.383840,4136.709867,97.60
